
| Dataset |
|------------|
| BoneMarrowA |
| BoneMarrowB |
| LungA |
| LungB |
| WholeBrainA |
| WholeBrainB |
| LargeIntestineA |
| LargeIntestineB |
| Cerebellum |
| PreFrontalCortex |
| SmallIntestine |

In [ ]:
import pandas as pd
import scipy.io
import anndata as ad
import numpy as np
import os

# Input file paths (change as needed)
cell_metadata_path = "/home/daozhang/Data/raw/GEO_data/mouse-atac/cell_metadata.txt"  # Contains cell metadata
peaks_path = "/home/daozhang/Data/raw/GEO_data/mouse-atac/atac_matrix.binary.qc_filtered.peaks.txt"  # Contains peak information
cells_path = "/home/daozhang/Data/raw/GEO_data/mouse-atac/atac_matrix.binary.qc_filtered.cells.txt"  # Contains all cells list
matrix_path = "/home/daozhang/Data/raw/GEO_data/mouse-atac/atac_matrix.binary.qc_filtered.mtx.gz"  # Contains sparse matrix file path

# Output directory for h5ad files (change as needed)
output_dir = "./Temp"
os.makedirs(output_dir, exist_ok=True)

# Tissue mapping dictionary (original tissue.replicate to dataset name)
tissue_mapping = {
    #"BoneMarrow_62016": "BoneMarrowA",
    "BoneMarrow_62216": "BoneMarrowB",
    # "Lung1_62216": "LungA",
    # "Lung2_62216": "LungB",
    # "WholeBrainA_62216": "WholeBrainA",
    # "WholeBrainA_62816": "WholeBrainB",
    # "LargeIntestineA_62816": "LargeIntestineA",
    # "LargeIntestineB_62816": "LargeIntestineB",
    # "Cerebellum_62216": "Cerebellum",
    # "PreFrontalCortex_62216": "PreFrontalCortex"
    # "SmallIntestine_62816": "SmallIntestine",
}

# Step 1: Read cell metadata
print("Reading cell metadata...")
cell_metadata = pd.read_csv(cell_metadata_path, sep="\t")

# Step 2: Read all cells list
print("Reading cells list...")
cells = pd.read_csv(cells_path, header=None, names=["cell"])

# Step 3: Read peak data
print("Reading peak information...")
peaks = pd.read_csv(peaks_path, header=None, names=["peak"])

# Step 4: Parse peak information
print("Parsing peak information...")
peaks_split = peaks["peak"].str.split("_", expand=True)
peaks_split.columns = ["chrom", "start", "end"]
peaks_split["chromStart"] = peaks_split["start"]
peaks_split["chromEnd"] = peaks_split["end"]
peaks_split["chr"] = peaks_split["chrom"]

# Step 5: Load sparse matrix
print("Loading sparse matrix...")
matrix = scipy.io.mmread(matrix_path).tocsr()
matrix = matrix.transpose()

# Process each tissue separately
for original_tissue, dataset_name in tissue_mapping.items():
    print(f"Processing {original_tissue} -> {dataset_name}...")
    
    # Filter metadata for the current tissue
    filtered_metadata = cell_metadata[cell_metadata["tissue.replicate"] == original_tissue]
    
    # Skip if no cells found for this tissue
    if filtered_metadata.empty:
        print(f"No cells found for {original_tissue}, skipping...")
        continue
    
    # Add Batch information based on dataset name
    filtered_metadata.loc[:, "Batch"] = dataset_name
    
    # Get selected cells and their indices
    selected_cells = filtered_metadata["cell"].tolist()
    filtered_cells_idx = cells[cells["cell"].isin(selected_cells)].index
    
    # Filter matrix for selected cells
    filtered_matrix = matrix[filtered_cells_idx, :]
    
    # Calculate n_cells for each peak (number of cells with accessible chromatin)
    print(f"Calculating n_cells for {dataset_name}...")
    n_cells_per_peak = (filtered_matrix > 0).sum(axis=0).A1  # Count non-zero values per peak
    
    # Add n_cells to peak information
    peaks_split_copy = peaks_split.copy()
    peaks_split_copy["n_cells"] = n_cells_per_peak
    
    # Construct var table
    var = peaks_split_copy[["chrom", "chromStart", "chromEnd", "chr", "start", "end", "n_cells"]]
    
    # Create AnnData object
    adata = ad.AnnData(X=filtered_matrix)
    
    # Add obs (cell metadata)
    adata.obs = filtered_metadata.set_index("cell")
    
    # Add var (peak data)
    # 使用原始峰名称作为索引
    adata.var = var
    adata.var.index = peaks["peak"]  # 使用原始的 "chr_start_end" 格式
    
    # Add CellType column (copy from cell_label)
    adata.obs["CellType"] = adata.obs["cell_label"]
    
    adata.obs["batch"] = 0  # Default to 0
    
    # Add n_genes column (count of non-zero features per cell)
    adata.obs["n_genes"] = (adata.X > 0).sum(axis=1).A1
    
    # Print some statistics
    print(f"Dataset {dataset_name} statistics:")
    print(f"  - Number of cells: {adata.n_obs}")
    print(f"  - Number of peaks: {adata.n_vars}")
    print(f"  - Average peaks per cell: {adata.obs['n_genes'].mean():.1f}")
    print(f"  - Average cells per peak: {adata.var['n_cells'].mean():.1f}")
    print(f"  - Peak accessibility range: {adata.var['n_cells'].min()} - {adata.var['n_cells'].max()} cells")
    
    # Save AnnData object
    output_file = os.path.join(output_dir, f"{dataset_name}.h5ad")
    print(f"Saving {dataset_name} dataset to {output_file}...")
    adata.write(output_file)
    
    print(f"Completed processing {dataset_name} with {len(filtered_cells_idx)} cells\n")

print("All tissue-specific h5ad files have been generated!")

| Dataset |
|------------|
| MosA1 |
| MosA2 |
| MosM1 |
| MosM2 |
| MosP1 |
| MosP2 |

In [ ]:
import h5py
import numpy as np
import scipy.sparse as sp
import scanpy as sc
import pandas as pd
from scipy.sparse import csr_matrix

def process_snap_file(file_path, batch_name, csv_file, output_dir):
    """
    Process SNAP (Single Nucleus ATAC-seq) HDF5 file and convert to AnnData format
    
    Parameters:
    -----------
    file_path : str
        Path to the input HDF5 file
    batch_name : str
        Batch identifier for the dataset
    csv_file : str
        Path to CSV file containing barcode to celltype mapping
    output_dir : str
        Directory to save the output H5AD file
    """
    
    # Open HDF5 file and extract data
    with h5py.File(file_path, 'r') as f:
        # Get PM (Peak Matrix) group data
        pm_group = f['PM']
        
        # Read sparse matrix components
        count_data = pm_group['count'][:]    # Non-zero element values
        idx = pm_group['idx'][:]             # Row indices of non-zero elements
        idy = pm_group['idy'][:]             # Column indices of non-zero elements
        
        # Get cell and peak information
        cell_barcodes = list(f['BD']['name'][:])          # Cell barcodes
        peak_chrom = list(f['PM']['peakChrom'][:])        # Peak chromosome names
        peak_start = list(f['PM']['peakStart'][:])        # Peak start positions
        peak_end = list(f['PM']['peakEnd'][:])            # Peak end positions
        
        # Convert byte strings to regular strings
        cell_barcodes = [name.decode('utf-8') for name in cell_barcodes]
        peak_chrom = [chrom.decode('utf-8') for chrom in peak_chrom]
        
        # Convert indices from 1-based to 0-based indexing
        idx = idx - 1
        idy = idy - 1
        
        # Ensure indices are integer type (required for sparse matrices)
        idx = idx.astype(int)
        idy = idy.astype(int)
        
        # Create sparse matrix in COO format
        count_matrix = sp.coo_matrix((count_data, (idx, idy)))
    
    # Create AnnData object
    print("Creating AnnData object...")
    adata = sc.AnnData(count_matrix)
    
    # Set cell and peak annotations
    adata.obs['cell'] = cell_barcodes
    adata.var['chr'] = peak_chrom
    adata.var['start'] = peak_start
    adata.var['end'] = peak_end
    
    # Load barcode to celltype mapping
    print("Loading cell type annotations...")
    barcode_df = pd.read_csv(csv_file)
    
    # Ensure barcode columns are string type
    barcode_df['barcode'] = barcode_df['barcode'].astype(str)
    adata.obs['cell'] = adata.obs['cell'].astype(str)
    
    # Create barcode to celltype mapping dictionary
    barcode_to_celltype = dict(zip(barcode_df['barcode'], barcode_df['celltype']))
    
    # Map cell types to observations
    adata.obs['CellType'] = adata.obs['cell'].map(barcode_to_celltype)
    adata.obs['Batch'] = batch_name
    
    # Convert sparse matrix to CSR format for efficiency
    adata.X = csr_matrix(adata.X)
    
    # Save processed data
    output_path = f"{output_dir}/{batch_name}.h5ad"
    print(f"Saving processed data to {output_path}")
    adata.write(output_path)
    
    print(f"Processing complete! AnnData shape: {adata.shape}")
    return adata


# Define file paths and parameters (change as needed)
# GSM3611836 snATAC_MOs_A_rep1
# GSM3611837 snATAC_MOs_A_rep2
# GSM3611838 snATAC_MOs_M_rep1
# GSM3611839 snATAC_MOs_M_rep2
# GSM3611840 snATAC_M0s_P_rep1
# GSM3611841 snATAC_MOs_P_rep2


# Define file paths and parameters (change as needed)
file_path = "/home/daozhang/Data/raw/GEO_data/GSE126724/GSM3611840_CEMBA180308_3B.snap.hdf5"
batch_name = "MosP1"
csv_file = "/home/daozhang/Data/raw/GEO_data/GSE126724/41467_2021_21583_MOESM4_ESM.csv"
output_dir = "./Temp"

# Process the data
adata = process_snap_file(file_path, batch_name, csv_file, output_dir)

| Dataset |
|------------|
| KidneyA |
| KidneyB |
| KidneyC |
| KidneyD |
| KidneyE |

In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np

# Sample UUID to batch name mapping (5 samples)
sample_uuid_mapping = {
    '8628f96c-2d97-4579-951c-044017ef6f0e': 'KidneyA',
    '84824920-47bc-4929-b123-87574944aa37': 'KidneyB',
    'c47a5959-a0ed-4276-8a8d-5f136f1edd29': 'KidneyC', 
    'd8b737fb-8a38-4a1d-a421-cbc81edddd05': 'KidneyD',
    'e7372715-150e-4117-a11b-a88ea56ce5c5': 'KidneyE'
}

def process_anndata_to_h5ad(adata):
    """
    Process AnnData object to generate standardized h5ad file
    """
    # Create a copy of AnnData
    adata_processed = adata.copy()
    
    # 1. Process obs data - keep only essential columns
    # Create new Batch column based on sample_uuid mapping
    adata_processed.obs['Batch'] = adata_processed.obs['sample_uuid'].map(sample_uuid_mapping)
    
    # Add CellType column (using existing cell_type column)
    adata_processed.obs['CellType'] = adata_processed.obs['cell_type']
    
    # Keep only essential obs columns
    essential_obs_cols = ['Batch', 'CellType']
    adata_processed.obs = adata_processed.obs[essential_obs_cols]
    
    # 2. Process var data - preserve genomic region information
    # Rename var columns to match requirements
    var_mapping = {
        'chrom': 'chr',
        'chromStart': 'start', 
        'chromEnd': 'end'
    }
    
    # Rename columns
    adata_processed.var = adata_processed.var.rename(columns=var_mapping)
    
    # Keep only essential var columns
    essential_var_cols = ['chr', 'start', 'end']
    adata_processed.var = adata_processed.var[essential_var_cols]
    
    # 3. Filter peaks - remove peaks accessible in <1% of cells
    # Calculate peak accessibility across cells
    peak_accessibility = (adata_processed.X > 0).sum(axis=0)
    if hasattr(peak_accessibility, 'A1'):  # Handle sparse matrix
        peak_accessibility = peak_accessibility.A1
    
    # Calculate threshold (1% of cells)
    min_cells_threshold = int(0.01 * adata_processed.n_obs)
    
    # Create filter mask
    peaks_to_keep = peak_accessibility >= min_cells_threshold
    
    print(f"Total peaks: {adata_processed.n_vars}")
    print(f"Min cells threshold (1%): {min_cells_threshold}")
    print(f"Peaks retained after filtering: {np.sum(peaks_to_keep)}")
    print(f"Peaks filtered out: {np.sum(~peaks_to_keep)}")
    
    # Apply filtering
    adata_processed = adata_processed[:, peaks_to_keep]
    
    # 4. Clean unnecessary metadata
    # Clean obsm (keep only essential)
    if 'X_umap' in adata_processed.obsm:
        adata_processed.obsm = {'X_umap': adata_processed.obsm['X_umap']}
    else:
        adata_processed.obsm = {}
    
    # Clean uns (keep only basic info)
    essential_uns = {}
    if 'title' in adata_processed.uns:
        essential_uns['title'] = adata_processed.uns['title']
    adata_processed.uns = essential_uns
    
    return adata_processed



# Load data (change as needed)
adata = sc.read_h5ad('/home/daozhang/Data/raw/Muto-2021-ATAC.h5ad') 
# Process data
adata_processed = process_anndata_to_h5ad(adata)
# Save processed file
adata_processed.write_h5ad('./Temp')


| Dataset |
|------------|
| MouseBrain(10X) |
| NormalCortex |


In [ ]:
import pandas as pd
import numpy as np
import anndata as ad
from scipy.sparse import csr_matrix
from scipy.io import mmread
import os

def create_and_filter_anndata(input_dir, output_file):
    """
    Create and filter AnnData object from files in the specified directory
    
    Parameters:
    input_dir (str): Directory containing input files
    output_file (str): Path to save the filtered h5ad file
    
    Returns:
    anndata.AnnData: Filtered AnnData object
    """
    # Define file paths
    barcode_file = os.path.join(input_dir, 'barcodes.tsv')
    peaks_file = os.path.join(input_dir, 'peaks.bed')
    matrix_file = os.path.join(input_dir, 'matrix.mtx')
    
    # Read barcodes
    with open(barcode_file, 'r') as f:
        barcodes = [line.strip() for line in f]
    
    # Read peaks
    peaks = pd.read_csv(peaks_file, sep='\t', header=None,
                       names=['chr', 'start', 'end'])
    
    # Read sparse matrix using mmread and transpose it
    data = mmread(matrix_file).tocsr().T
    
    # Calculate n_genes (number of non-zero peaks per cell)
    n_genes = np.asarray(data.sum(axis=1)).flatten()
    
    # Calculate n_cells (number of cells with non-zero counts per peak)
    n_cells = np.asarray(data.sum(axis=0)).flatten()
    
    # Create obs DataFrame
    obs = pd.DataFrame(index=barcodes)
    obs['cell'] = barcodes
    obs['tissue'] = 'NormalCortex'
    obs['Batch'] = 'NormalCortex'
    obs['batch'] = ['1'] * len(barcodes)
    obs['n_genes'] = n_genes
    
    # Create var DataFrame with formatted peak names
    var = pd.DataFrame(index=range(len(peaks)))
    var['chr'] = peaks['chr']
    var['start'] = peaks['start']
    var['end'] = peaks['end']
    var['n_cells'] = n_cells
    
    # Create formatted peak names
    var.index = var.apply(lambda row: f"{row['chr']}_{row['start']}_{row['end']}", axis=1)
    
    # Create initial AnnData object
    adata = ad.AnnData(X=data,
                      obs=obs,
                      var=var)
    
    print("Initial AnnData:")
    print(f"Matrix shape: {data.shape}")
    print(f"Number of barcodes: {len(barcodes)}")
    print(f"Number of peaks: {len(peaks)}")
    print(adata)
    
    # Filter cells and peaks
    cells_mask = adata.obs['n_genes'] > 0
    peaks_mask = adata.var['n_cells'] > 0
    
    # Create filtered AnnData object
    adata_filtered = adata[cells_mask, peaks_mask].copy()
    
    print("\nFiltering results:")
    print(f"Removed {np.sum(~cells_mask)} cells with no expression")
    print(f"Removed {np.sum(~peaks_mask)} peaks with no expression")
    print(f"Final dimensions: {adata_filtered.shape}")
    
    # Save the filtered AnnData object
    adata_filtered.write_h5ad(output_file)
    print(f"\nSaved filtered AnnData to: {output_file}")
    
    return adata_filtered


# Set input and output paths (change as needed)
input_directory = "/home/daozhang/Data/raw/Normal_cortex"
output_file = "./Temp/NormalCortex.h5ad"

# Create and filter AnnData
adata = create_and_filter_anndata(input_directory, output_file)